In [78]:
import numpy as np
import pandas as pd
import functools as ft

In [79]:
df= pd.read_csv("E:/nyc-crime/NYPD_Complaint_Data_Historic(1).csv")

C:\Users\rajaa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18,20) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [117]:
df1= pd.read_csv("nycFreqSorted.csv")

In [80]:
toDrop= ['CMPLNT_NUM','CMPLNT_FR_DT','CMPLNT_FR_TM','CMPLNT_TO_DT','CMPLNT_TO_TM','ADDR_PCT_CD',
'KY_CD','PD_CD','PD_DESC','CRM_ATPT_CPTD_CD','LAW_CAT_CD',
'LOC_OF_OCCUR_DESC','PREM_TYP_DESC','JURIS_DESC','JURISDICTION_CODE','PARKS_NM','HADEVELOPT',
'HOUSING_PSA','X_COORD_CD','Y_COORD_CD','SUSP_AGE_GROUP','SUSP_RACE','SUSP_SEX','TRANSIT_DISTRICT',
'Latitude','Longitude','Lat_Lon','PATROL_BORO','STATION_NAME','VIC_AGE_GROUP','VIC_RACE','VIC_SEX']

df.drop(toDrop, inplace= True, axis= 1)

In [81]:
df= df.loc[df['BORO_NM'].notnull()]

In [82]:
borough= pd.unique(df['BORO_NM'])
crimes= pd.unique(df['OFNS_DESC'])

In [83]:
df['YEAR']= df.RPT_DT.str.slice(-4)
df['RPT_DT']= df.RPT_DT.str.slice(0,2)
df['RPT_DT']= df.RPT_DT.str.replace('/','')

df= df.astype({'RPT_DT': int, 'YEAR': int})

In [84]:
years= pd.unique(df['YEAR'])
months= pd.unique(df['RPT_DT'])

years.sort()
months.sort()

In [85]:
def getMultiIndex(yr,mon):
    l1= [yr[0]]*np.alen(mon)
    for i in yr:
        if i==yr[0]:
            continue
        l2= [i]*np.alen(mon)
        l1.extend(l2)
    m1= list(mon)
    for j in np.arange(np.alen(yr)-1):
        m1.extend(mon)

    arrays= [l1,m1]
    tuples= list(zip(*arrays))
    ind= pd.MultiIndex.from_tuples(tuples, name= ['year','month'])
    return ind

index= getMultiIndex(years,months)


In [86]:
df= df.rename(columns={'OFNS_DESC': 'Category', 'RPT_DT': 'Month', 'YEAR': 'Year'})

In [87]:
boroList= []
for i in borough:
    boroList.append(df.loc[df['BORO_NM']==i])

In [138]:
boroFreq=[]

In [139]:
#to retrieve data frame for all years on monthly basis crime frequency
def getYearDataFrame(NoOfYears,boroDF5):
    yearList= list(np.arange(NoOfYears))
    for i in np.arange(NoOfYears):
        yearList[i]= yearDataFrame(years[i],boroDF5)
    return yearList

#to retrieve data frame for monthly basis crime freqeuncy for a single year on the basis of the given year
def yearDataFrame(yearNumber,boroDF1):
    yearTemp= boroDF1.loc[boroDF1['Year']==yearNumber]
    monthFrame= getMonthDataFrame(yearTemp,len(months),yearNumber,boroDF1)
    yearDf= ft.reduce(lambda left, right: pd.merge(left,right,on='Category',how='outer'),monthFrame)
    yearDf=yearDf.fillna(0)
    return yearDf

#to retrive the monthly basis crime frequency for all months of a year
def getMonthDataFrame(yearTem,noOfMonths,yearNum,boroDF2):
    monthList= list(np.arange(noOfMonths))
    for i in np.arange(noOfMonths):
        monthList[i]= monthDataFrame(yearTem,months[i],yearNum,boroDF2)
    return monthList

#to retrieve the dataframe for crime frequency of a single month
def monthDataFrame(yearTempo,monthNumber,yearNo,boroDF3):
    monthTemp= yearTempo.loc[boroDF3['Month']==monthNumber]
    monthlyTempDf= monthTemp['Category'].value_counts().rename_axis('Category').reset_index(name="Month-"+str(monthNumber)+"("+str(yearNo)+")")
    return monthlyTempDf    

def getDF(boroDF):
    final_list= getYearDataFrame(len(years),boroDF)
    #merging all the year based dataframe into a single dataframe taking the Category as the common field
    df_final= ft.reduce(lambda left, right: pd.merge(left,right,on='Category',how='outer'),final_list)

    #replace all na values with 0
    df_final= df_final.fillna(0)
    return df_final


def getForAllBorough(boro):
    for i in boro:
        boroFreq.append(getDF(i))

getForAllBorough(boroList)

In [141]:
for k in np.arange(np.alen(boroFreq)):
    #adding row showing total crime in each month
    crimeInMonth= np.arange((len(months)*len(years))+1)
    count=0
    for i in np.arange(len(years)):
         for j in np.arange(len(months)):
               #crimeInMonth[count+1]= np.sum(df_final['Month-12(2015)'])
                crimeInMonth[count+1]= np.sum(boroFreq[k]['Month-'+str(months[j])+"("+str(years[i])+')'])
                count+=1
               # crimeInMonth[count+1]= np.sum(df_final['Month-'+str(months[j])+"("+str(years[i])+")"])
    

    crimeMonth= pd.DataFrame([crimeInMonth],columns=boroFreq[k].columns)
    crimeMonth['Category']='TOTAL'

    boroFreq[k]= pd.concat([boroFreq[k],crimeMonth],ignore_index=True)

In [143]:
categoryList= []
data=[]
subDfList=[]

for i in np.arange(np.alen(boroFreq)):
    cat= boroFreq[i]['Category']
    boroFreq[i].drop(['Category'],inplace= True, axis=1)
    data.append(boroFreq[i].values)
    subDfList.append(pd.DataFrame(data= data[i], columns= index))
    subDfList[i].insert(0,'Category',cat)
    subDfList[i]= subDfList[i].set_index('Category')

In [105]:
for i in np.arange(np.alen(subDfList)):
    #to Download the mutated dataframe as csv
    subDfList[i].to_csv(borough[i]+"_Frequency.csv")

In [220]:
subDfPercentList=[]

In [222]:
# a function to calculate percentage of each crime in each month per year

def getPercentDataFrame(df2):
    num=0
    dfTest= df2
    monTotal= dfTest.last_valid_index()
    for i in years:
        for j in months:
            num=num+1
            if dfTest[i][j][monTotal]>0:
                dfTest[i][j]*=(100/df1.iloc[73][num])
                continue
            dfTest[i][j]=0
            
    return dfTest
#function call
for k in subDfList:
     subDfPercentList.append(getPercentDataFrame(k))


<ipython-input-222-da305a4d7527>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTest[i][j]*=(100/df1.iloc[73][num])


In [226]:
for i in np.arange(np.alen(subDfPercentList)):
    #to Download the mutated dataframe as csv
    subDfPercentList[i].to_csv(borough[i]+"_PercentageTotal.csv")